# 接口

接口定义了一组方法（方法集），但是这些方法不包含（实现）代码。顾名思义，它就是个 “接口”。

通过如下格式定义接口类型：

```
type 接口名 interface {
    方法1(参数列表) 返回值类型
    方法2(参数列表) 返回值类型
    ...
}
```

类型（比如结构体）可以实现某个接口，这个实现可以描述为，该类型的变量上的每一个具体方法所组成的集合，包含了该接口的方法集：

`类型的接口的方法集 ⊂ 类型的方法集`

只要类型实现了接口中的方法，它就实现了此接口。下面是接口的例子：

In [5]:
// 定义一个结构类型 Square
type Square struct {
    side float32
}

In [6]:
// Square 实现了方法 Area
func (sq *Square) Area() float32 {
    return sq.side * sq.side
}

In [7]:
// 定义一个接口
type Shaper interface {
    Area() float32
}

类型 Square 实现了接口 Shaper 中的方法，所以它实现了接口 Shaper：

In [16]:
sq := new(Square) // 创建一个类型的实例 sq
sq.side = 5

// 实例赋给接口，等价于 sp1 := Shaper(sq)
// sp1 := Shaper(sq)
var sp1 Shaper
sp1 = sq

sp1.Area() // 调用接口方法

25

一个类型可以实现多个接口。

多个类型可以实现同一个接口：

In [17]:
type myfloat float32

func (m myfloat) Area() float32 {
    return float32(m * m)
}

x := myfloat(5)
sp2 := Shaper(x)
sp2.Area() // 调用接口方法

25

所有类型都实现了空接口 `interface{}` 类型：

In [9]:
type Emptyer interface{}
Emptyer(1)

1

In [10]:
Emptyer(true)

true

接口类型可以显式指定方法，也可以通过接口类型名称嵌入其他接口的方法。此时接口的方法集包含嵌入接口的方法：

    type Reader interface {
            Read(p []byte) (n int, err error)
            Close() error
    }

    type Writer interface {
            Write(p []byte) (n int, err error)
            Close() error
    }

    // ReadWriter's methods are Read, Write, and Close.
    type ReadWriter interface {
            Reader  // includes methods of Reader in ReadWriter's method set
            Writer  // includes methods of Writer in ReadWriter's method set
    }

接口中每个显式指定的方法的名称必须是非空白（`_`）且唯一的。

**类型断言**：

一个接口类型的变量 `x` 中可以包含任何类型的值，必须有一种方式来检测它的 *动态* 类型，即运行时在变量中存储的值的实际类型。在执行过程中动态类型可能会有所不同，但是它总是可以分配给接口变量本身的类型。通常我们可以使用 类型断言 来测试在某个时刻 `x` 是否包含类型 `T` 的值。

类型断言表达式：

    x.(T)
    
断言的结果为 `(类型T的值, bool)` 的值对。若类型断言成立，表达式的值即为 `x` 转换到类型 `T` 的值和 `true`，否则表达式的值是类型 `T` 的零值和 `false`：

In [8]:
import "math"

type Square struct {
    side float32
}

type Circle struct {
    radius float32
}

type Shaper interface {
    Area() float32
}

func (sq *Square) Area() float32 {
    return sq.side * sq.side
}

func (ci *Circle) Area() float32 {
    return ci.radius * ci.radius * math.Pi
}

sq1 := new(Square)
sq1.side = 5

x := Shaper(sq1)

In [9]:
x.(*Square) // 成功

&{5} true

In [10]:
x.(*Circle) // 失败

<nil> false

类型断言可能是无效的，虽然编译器会尽力检查转换是否有效，但是它不可能预见所有的可能性。如果转换在程序运行时失败会导致错误发生。更安全的方式是使用类似以下形式来进行类型断言：

    if v, ok := varI.(T); ok {
        Process(v)
        return
    }